In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
train_df = pd.read_csv('./new_train.csv', parse_dates=["startdate"])
test_df = pd.read_csv('./new_test.csv', parse_dates=["startdate"])

In [3]:
exclude_cols = ['index', 'startdate']
temporal_attrs = ['year', 'month', 'day_of_year', 'season', 'day_of_year_sin', 'day_of_year_cos','month_sin', 'month_cos', 'season_sin', 'season_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in temporal_attrs and c not in loc_attrs and c not in target and c not in embedding_attrs]
print("Main features:", len(main_attrs))

Main features: 240


In [4]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-wind-h100-14d__wind-hgt-100',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'month_sin',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'nmme-prate-34w__gfdlflora',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13']

# important_cols = ['contest-wind-h500-14d__wind-hgt-500']

In [5]:
train_df.columns

Index(['index', 'lat', 'lon', 'startdate',
       'contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-tmp2m-34w__cancm30',
       'nmme0-tmp2m-34w__cancm40', 'nmme0-tmp2m-34w__ccsm30',
       'nmme0-tmp2m-34w__ccsm40', 'nmme0-tmp2m-34w__cfsv20',
       ...
       'year', 'month', 'day_of_year', 'season', 'day_of_year_sin',
       'day_of_year_cos', 'month_sin', 'month_cos', 'season_sin',
       'season_cos'],
      dtype='object', length=257)

In [7]:
# train_df['resendation'] = train_df['contest-pevpr-sfc-gauss-14d__pevpr'] * train_df['contest-rhum-sig995-14d__rhum'] / 100
# test_df['resendation'] = test_df['contest-pevpr-sfc-gauss-14d__pevpr'] * test_df['contest-rhum-sig995-14d__rhum'] / 100

In [12]:
# train_df = train_df.drop(columns = ['resendation'])
# test_df = test_df.drop(columns = ['resendation'])

In [23]:
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

X1, Y1 = train_df[best_cols].copy(), train_df[target[0]].copy()

from catboost import Pool, cv, CatBoostRegressor
from bayes_opt import BayesianOptimization
from bayes_opt import BayesianOptimization as BO
import warnings
from sklearn.model_selection import * 
from sklearn.metrics import *

Use_BO = True

if Use_BO:
    def CB_opt(depth, learning_rate, l2_leaf_reg, model_size_reg): 

        scores = []
        trainx, valx, trainy, valy = train_test_split(X1, Y1, test_size=0.33, random_state=42)

        reg = CatBoostRegressor(   
                                        verbose = 0,
                                        learning_rate = learning_rate,
                                        l2_leaf_reg = l2_leaf_reg,
                                        task_type="GPU",
                                        devices='0:1',
                                        max_depth = int(depth),
                                        random_state = 1212,
                                        bootstrap_type =  'Bayesian',
                                        use_best_model = True, 
                                        loss_function='RMSE',
                                        model_size_reg = model_size_reg
                                    )

        reg.fit(trainx, trainy, eval_set = (valx, valy))
        y_pred = reg.predict(valx)
        scores.append(rmse(valy, y_pred))

        return 1/np.mean(scores)

    #"n_estimators": (150,1200),
    # "num_leaves": (100,150),
    # "max_bin":(150,300),
    pbounds = {
               "depth": (6, 7),
               "learning_rate": (0.09, 0.0980689972639084),
               "l2_leaf_reg":(2,4),
               "model_size_reg": (0.48, 0.5)
    }

    optimizer = BayesianOptimization(f = CB_opt, pbounds = pbounds,  verbose = 2, random_state = 1212)

    optimizer.maximize(init_points = 7, n_iter = 30, acq = 'ucb', alpha = 1e-6)

    print(optimizer.max)

    max_bo_params = optimizer.max['params']

    max_bo_params

|   iter    |  target   |   depth   | l2_lea... | learni... | model_... |
-------------------------------------------------------------------------


/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument



| 1         | 1.624     | 6.245     | 3.609     | 0.09141   | 0.4994    |


| 2         | 1.621     | 6.225     | 3.738     | 0.09227   | 0.4945    |


| 3         | 1.645     | 6.752     | 2.493     | 0.09623   | 0.4817    |


...

| 33        | 1.641     | 6.764     | 2.173     | 0.09363   | 0.4882    |


| 34        | 1.643     | 6.094     | 2.895     | 0.09536   | 0.4927    |


| 35        | 1.617     | 6.406     | 3.131     | 0.09086   | 0.4956    |


| 36        | 1.652     | 6.409     | 3.124     | 0.09475   | 0.4875    |


| 37        | 1.631     | 6.451     | 3.657     | 0.09324   | 0.4838    |
{'target': 1.6607332772061196, 'params': {'depth': 6.807269541146839, 'l2_leaf_reg': 2.7793361781124255, 'learning_rate': 0.09761708892270968, 'model_size_reg': 0.4803376984686182}}


In [25]:
Use_BO_result = True

if Use_BO_result:
    opt_params = {
              'iterations':2000,
              'verbose':0,
              'learning_rate' : max_bo_params['learning_rate'],
              'l2_leaf_reg' : max_bo_params['l2_leaf_reg'],
              'max_depth' : int(max_bo_params['depth']), 
              'use_best_model' : True, 
              'loss_function' : 'RMSE',
                'task_type': "GPU",
                                        'devices':'0:1',
              'model_size_reg' : max_bo_params['model_size_reg']
             }
else:
    opt_params = {
          'iterations':2000,
          'verbose':0,
          'learning_rate' : 0.0980689972639084,
          'subsample' : 0.7443133148363695, 
          'l2_leaf_reg' : 2.3722386345448316,
          'max_depth' : int(6.599144674342465),
          'use_best_model' : True, 
          'loss_function' : 'RMSE',
          'model_size_reg' : 0.4833187897595954
         }

In [8]:
import seaborn as sns
def plot_distribution(var):
    plt.subplots(figsize=(14,7))
    sns.distplot(x=train_df[var], color='blue', kde=True)
    sns.distplot(x=test_df[var], color='red', kde=True)
    plt.title(var, weight="bold",fontsize=20, pad=20)
    plt.ylabel("Count", weight="bold", fontsize=15)
    plt.xlabel(var, weight="bold", fontsize=12)
    plt.show()

In [29]:
X = train_df[best_cols].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1', learning_rate = 0.0980689972639084,
              l2_leaf_reg = 2.3722386345448316,
              max_depth = 6, 
              use_best_model = True, 
              loss_function = 'RMSE',
              model_size_reg = 0.4833187897595954)
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[best_cols].values))
s = 0
for pred in preds:
    s += pred
s = s/5
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_corr_important_v5.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:01,  2.95it/s]


(375734, 62) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 8115.5625 Total: 24268.3125


0:	learn: 9.0473215	test: 9.0256015	best: 9.0256015 (0)	total: 224ms	remaining: 18m 40s
1:	learn: 8.3021525	test: 8.2823581	best: 8.2823581 (1)	total: 230ms	remaining: 9m 33s
2:	learn: 7.6359974	test: 7.6181827	best: 7.6181827 (2)	total: 234ms	remaining: 6m 29s
3:	learn: 7.0327265	test: 7.0169007	best: 7.0169007 (3)	total: 238ms	remaining: 4m 57s
4:	learn: 6.4935420	test: 6.4796354	best: 6.4796354 (4)	total: 242ms	remaining: 4m 1s
5:	learn: 6.0054297	test: 5.9934012	best: 5.9934012 (5)	total: 246ms	remaining: 3m 24s
6:	learn: 5.5697564	test: 5.5601026	best: 5.5601026 (6)	total: 250ms	remaining: 2m 58s
7:	learn: 5.1817185	test: 5.1728702	best: 5.1728702 (7)	total: 254ms	remaining: 2m 38s
8:	learn: 4.8350024	test: 4.8268428	best: 4.8268428 (8)	total: 258ms	remaining: 2m 23s
9:	learn: 4.5254892	test: 4.5183300	best: 4.5183300 (9)	total: 262ms	remaining: 2m 10s
10:	learn: 4.2324523	test: 4.2261978	best: 4.2261978 (10)	total: 266ms	remaining: 2m
11:	learn: 3.9734543	test: 3.9677203	best: 3.

128:	learn: 1.2681613	test: 1.2703871	best: 1.2703871 (128)	total: 826ms	remaining: 31.2s
129:	learn: 1.2655733	test: 1.2679665	best: 1.2679665 (129)	total: 830ms	remaining: 31.1s
130:	learn: 1.2628716	test: 1.2652754	best: 1.2652754 (130)	total: 834ms	remaining: 31s
131:	learn: 1.2607183	test: 1.2630347	best: 1.2630347 (131)	total: 842ms	remaining: 31s
132:	learn: 1.2579974	test: 1.2604221	best: 1.2604221 (132)	total: 849ms	remaining: 31.1s
133:	learn: 1.2555282	test: 1.2579541	best: 1.2579541 (133)	total: 853ms	remaining: 31s
134:	learn: 1.2533111	test: 1.2557970	best: 1.2557970 (134)	total: 861ms	remaining: 31s
135:	learn: 1.2514122	test: 1.2539026	best: 1.2539026 (135)	total: 869ms	remaining: 31.1s
136:	learn: 1.2492732	test: 1.2517927	best: 1.2517927 (136)	total: 873ms	remaining: 31s
137:	learn: 1.2474257	test: 1.2499078	best: 1.2499078 (137)	total: 880ms	remaining: 31s
138:	learn: 1.2445290	test: 1.2470866	best: 1.2470866 (138)	total: 887ms	remaining: 31s
139:	learn: 1.2423952	te

...

4670:	learn: 0.3007363	test: 0.3311354	best: 0.3311354 (4670)	total: 22.9s	remaining: 1.61s
4671:	learn: 0.3006958	test: 0.3311030	best: 0.3311030 (4671)	total: 22.9s	remaining: 1.61s
4672:	learn: 0.3006774	test: 0.3310852	best: 0.3310852 (4672)	total: 22.9s	remaining: 1.6s
4673:	learn: 0.3006515	test: 0.3310587	best: 0.3310587 (4673)	total: 22.9s	remaining: 1.6s
4674:	learn: 0.3006200	test: 0.3310283	best: 0.3310283 (4674)	total: 22.9s	remaining: 1.59s
4675:	learn: 0.3005972	test: 0.3310087	best: 0.3310087 (4675)	total: 22.9s	remaining: 1.59s
4676:	learn: 0.3005698	test: 0.3309848	best: 0.3309848 (4676)	total: 22.9s	remaining: 1.58s
4677:	learn: 0.3005366	test: 0.3309639	best: 0.3309639 (4677)	total: 22.9s	remaining: 1.58s
4678:	learn: 0.3005027	test: 0.3309306	best: 0.3309306 (4678)	total: 22.9s	remaining: 1.57s
4679:	learn: 0.3004843	test: 0.3309105	best: 0.3309105 (4679)	total: 23s	remaining: 1.57s
4680:	learn: 0.3004535	test: 0.3308922	best: 0.3308922 (4680)	total: 23s	remaining: 

4792:	learn: 0.2968860	test: 0.3275836	best: 0.3275836 (4792)	total: 23.5s	remaining: 1.01s
4793:	learn: 0.2968605	test: 0.3275655	best: 0.3275655 (4793)	total: 23.5s	remaining: 1.01s
4794:	learn: 0.2968278	test: 0.3275368	best: 0.3275368 (4794)	total: 23.5s	remaining: 1s
4795:	learn: 0.2968001	test: 0.3275132	best: 0.3275132 (4795)	total: 23.5s	remaining: 1s
4796:	learn: 0.2967798	test: 0.3274967	best: 0.3274967 (4796)	total: 23.5s	remaining: 996ms
4797:	learn: 0.2967582	test: 0.3274717	best: 0.3274717 (4797)	total: 23.5s	remaining: 991ms
4798:	learn: 0.2967283	test: 0.3274472	best: 0.3274472 (4798)	total: 23.5s	remaining: 986ms
4799:	learn: 0.2966974	test: 0.3274112	best: 0.3274112 (4799)	total: 23.5s	remaining: 981ms
4800:	learn: 0.2966571	test: 0.3273778	best: 0.3273778 (4800)	total: 23.6s	remaining: 976ms
4801:	learn: 0.2966353	test: 0.3273572	best: 0.3273572 (4801)	total: 23.6s	remaining: 971ms
4802:	learn: 0.2966174	test: 0.3273444	best: 0.3273444 (4802)	total: 23.6s	remaining: 

4909:	learn: 0.2935669	test: 0.3246118	best: 0.3246118 (4909)	total: 24.1s	remaining: 442ms
4910:	learn: 0.2935437	test: 0.3246007	best: 0.3246007 (4910)	total: 24.1s	remaining: 437ms
4911:	learn: 0.2935234	test: 0.3245769	best: 0.3245769 (4911)	total: 24.1s	remaining: 432ms
4912:	learn: 0.2934999	test: 0.3245583	best: 0.3245583 (4912)	total: 24.1s	remaining: 427ms
4913:	learn: 0.2934646	test: 0.3245345	best: 0.3245345 (4913)	total: 24.1s	remaining: 423ms
4914:	learn: 0.2934390	test: 0.3245100	best: 0.3245100 (4914)	total: 24.1s	remaining: 418ms
4915:	learn: 0.2934063	test: 0.3244865	best: 0.3244865 (4915)	total: 24.2s	remaining: 413ms
4916:	learn: 0.2933844	test: 0.3244537	best: 0.3244537 (4916)	total: 24.2s	remaining: 408ms
4917:	learn: 0.2933689	test: 0.3244429	best: 0.3244429 (4917)	total: 24.2s	remaining: 403ms
4918:	learn: 0.2933377	test: 0.3244088	best: 0.3244088 (4918)	total: 24.2s	remaining: 398ms
4919:	learn: 0.2933176	test: 0.3243986	best: 0.3243986 (4919)	total: 24.2s	remai

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 16.98it/s]


cos_sim with best submission: 0.9999493215710683
